In [1]:
using BenchmarkTools

In [2]:
function rotate(matrix)
    return reverse(reverse(matrix, dims=1), dims=2)
end

rotate (generic function with 1 method)

In [3]:
function Convolution_2d_pad(input, kernel; bias=0., padding=false)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    if padding
        padded_input = zeros(Float32, input_rows + 2*kernel_height - 2, input_columns + 2*kernel_width - 2)
        padded_input[kernel_height:end-kernel_height+1, kernel_width:end-kernel_width+1] .= input
        input_rows, input_columns = size(padded_input)
        input = padded_input
    end

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    output = zeros(Float32, output_rows, output_columns)


    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            output[r, c] = sum(patch .* kernel) + bias
        end
    end
    return output
end

Convolution_2d_pad (generic function with 1 method)

In [4]:
function Convolution_2d(input, kernel; bias=0.)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    output = zeros(Float32, output_rows, output_columns)


    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            output[r, c] = sum(patch .* kernel) + bias
        end
    end
    return output
end

Convolution_2d (generic function with 1 method)

In [5]:
function Convolution_2d!(ret, input, kernel; bias=0., padding=false)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    if padding
        padded_input = zeros(Float32, input_rows + 2*kernel_height - 2, input_columns + 2*kernel_width - 2)
        padded_input[kernel_height:end-kernel_height+1, kernel_width:end-kernel_width+1] .= input
        input_rows, input_columns = size(padded_input)
        input = padded_input
    end

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    sumret = zeros(size(kernel))
    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            sumret .= patch .* kernel
            ret[r, c] = sum(sumret) + bias
            sumret .= 0.0
        end
    end
end

Convolution_2d! (generic function with 1 method)

In [6]:
function Conv_backward_v1( input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    
    grad_input = zeros(Float32, size(input))
    for k in 1:input_channels
        for c in 1:output_channels
            grad_input[:, :, k] += Convolution_2d_pad(weights[:, :, k, c], gradient[:, :, c]; padding=true)
        end
    end

    grad_weights = zeros(Float32, size(weights))
    for k in 1:input_channels
        for c in 1:output_channels
            grad_weights[:, :, k, c] += Convolution_2d(input[:, :, k], gradient[:, :, c])
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v1 (generic function with 1 method)

In [7]:
function Conv_backward_v2(input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    
    grad_input = zeros(Float32, size(input))
    grad_weights = zeros(Float32, size(weights))
    
    tmp_weights = zeros(Float32, kernel_height, kernel_width)
    tmp_input = zeros(Float32, input_height, input_width)
    tmp_gradient = zeros(Float32, output_height, output_width)
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_weights .= @views weights[:, :, k, c]
            tmp_gradient .= @views gradient[:, :, c]
            grad_input[:, :, k] .+= Convolution_2d_pad(tmp_weights, rotate(tmp_gradient); padding=true)
        end
    end

    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_input .= @views input[:, :, k]
            tmp_gradient .= @views gradient[:, :, c]
            grad_weights[:, :, k, c] += Convolution_2d(tmp_input, tmp_gradient)
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v2 (generic function with 1 method)

In [8]:
function Conv_backward_v3( input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    grad_input = zeros(Float32, size(input))
    grad_weights = zeros(Float32, size(weights))
    
    tmp_weights = zeros(Float32, kernel_height, kernel_width)
    tmp_input = zeros(Float32, input_height, input_width)
    tmp_gradient = zeros(Float32, output_height, output_width)
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_weights .= @views weights[:, :, k, c]
            tmp_gradient .= @views gradient[:, :, c]
            for i = 1:output_height
                for j = 1:output_width
                    grad_input[i:i+kernel_height-1, j:j+kernel_width-1, k] .+= (tmp_weights .* tmp_gradient[i,j]);
                end
            end
        end
    end
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_input .= @views input[:, :, k]
            tmp_gradient .= @views gradient[:, :, c]
            grad_weights[:, :, k, c] += Convolution_2d(tmp_input, tmp_gradient)
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v3 (generic function with 1 method)

In [9]:
function Conv_backward_v4(input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    grad_input = zeros(Float32, size(input))
    grad_weights = zeros(Float32, size(weights))
    
    tmp_weights = zeros(Float32, kernel_height, kernel_width)
    tmp_input = zeros(Float32, input_height, input_width)
    tmp_gradient = zeros(Float32, output_height, output_width)
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_weights .= @views weights[:, :, k, c]
            tmp_gradient .= @views gradient[:, :, c]
            for i = 1:output_height
                for j = 1:output_width
                    grad_input[i:i+kernel_height-1, j:j+kernel_width-1, k] .+= (tmp_weights .* tmp_gradient[i,j]);
                end
            end
        end
    end
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_input .= @views input[:, :, k]
            tmp_gradient .= @views gradient[:, :, c]
            Convolution_2d!(tmp_weights, tmp_input, tmp_gradient)
            grad_weights[:, :, k, c] .+= tmp_weights
            tmp_weights .= 0
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v4 (generic function with 1 method)

In [26]:
function Conv_backward_v5(input, weights, bias, gradient)
  input_height, input_width, input_channels = size(input)
  output_height, output_width, output_channels = size(gradient)
  kernel_height, kernel_width, _, _ = size(weights)
  
  grad_input = zeros(Float32, size(input))
  grad_weights = zeros(Float32, size(weights))
  
  tmp_weights = zeros(Float32, kernel_height, kernel_width)
  tmp_input = zeros(Float32, input_height, input_width)
  tmp_gradient = zeros(Float32, output_height, output_width)
  tmp_weight_gradient = zeros(Float32, kernel_height, kernel_width)
  for k in 1:input_channels
      for c in 1:output_channels
          for i = 1:output_height
              for j = 1:output_width
                  @views grad_input[i:i+kernel_height-1, j:j+kernel_width-1, k] .+= (weights[:, :, k, c] .* gradient[i, j, c]);
              end
          end
      end
  end
  
  for k in 1:input_channels
      for c in 1:output_channels
          tmp_weights .= 0
          tmp_input .= @views input[:, :, k]
          tmp_gradient .= @views gradient[:, :, c]
          Convolution_2d!(tmp_weights, tmp_input, tmp_gradient)
          grad_weights[:, :, k, c] .+= tmp_weights
      end
  end

  grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
  
  return grad_input, grad_weights, grad_bias
end

Conv_backward_v5 (generic function with 1 method)

In [11]:
input = rand(28, 28, 1);
weights = rand(3, 3, 1, 6);
bias = rand(Float32, 6);
gradient = rand(26, 26, 6);

In [12]:
@benchmark Conv_backward_v1(input, weights, bias, gradient)

BenchmarkTools.Trial: 507 samples with 1 evaluation.
 Range (min … max):  3.044 ms … 34.869 ms  ┊ GC (min … max):  0.00% … 28.90%
 Time  (median):     8.282 ms              ┊ GC (median):    16.70%
 Time  (mean ± σ):   9.846 ms ±  5.656 ms  ┊ GC (mean ± σ):  17.33% ± 14.87%

  █▂ ▃▅▄█▃▂▂▃▂     ▁                                          
  ██▅███████████▆▅▆█▇██▅▅▇▇▆▄▄▃▃▃▃▅▄▃▄▂▁▃▄▂▁▁▂▂▃▂▃▁▁▁▂▂▁▁▁▁▂ ▄
  3.04 ms        Histogram: frequency by time        31.2 ms <

 Memory estimate: 25.49 MiB, allocs estimate: 4842.

In [13]:
@benchmark Conv_backward_v2(input, weights, bias, gradient)

BenchmarkTools.Trial: 936 samples with 1 evaluation.
 Range (min … max):  2.839 ms … 20.211 ms  ┊ GC (min … max):  0.00% … 57.36%
 Time  (median):     4.895 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   5.327 ms ±  2.384 ms  ┊ GC (mean ± σ):  16.69% ± 18.38%

  █▂                                                          
  ██▅▅▅▄▄▅▄▄▄▅▆▆▄▄▅▄▄▄▄▃▃▃▃▃▄▄▃▃▃▃▃▃▂▂▂▂▂▃▃▂▂▂▂▂▂▂▂▂▁▂▁▁▂▁▂▂ ▃
  2.84 ms        Histogram: frequency by time        13.1 ms <

 Memory estimate: 13.22 MiB, allocs estimate: 4827.

In [14]:
@benchmark Conv_backward_v3(input, weights, bias, gradient)

BenchmarkTools.Trial: 9511 samples with 1 evaluation.
 Range (min … max):  300.200 μs …   9.706 ms  ┊ GC (min … max): 0.00% … 96.35%
 Time  (median):     364.400 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   516.184 μs ± 402.558 μs  ┊ GC (mean ± σ):  3.43% ±  7.95%

  █▆▅▅▄▃▃▂▂▃▃▃▃▂▂▂▁▂▁▁▁▁                                        ▁
  ████████████████████████████▇▇▆▆█▆▆▅▅▆▅▆▆▅▄▆▅▅▅▄▅▄▄▅▄▄▄▅▄▄▃▄▄ █
  300 μs        Histogram: log(frequency) by time       2.18 ms <

 Memory estimate: 544.08 KiB, allocs estimate: 4149.

In [15]:
@benchmark Conv_backward_v4(input, weights, bias, gradient)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  304.000 μs …  38.811 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     347.150 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   455.687 μs ± 473.135 μs  ┊ GC (mean ± σ):  2.09% ± 6.42%

  █▆▅▅▄▄▃▂▂▁▂▂▂▂▃▂▂▂▂▂▁▁▁▁▁                                     ▁
  ███████████████████████████████▇▇▆▆▆▆▆▅▆▄▅▅▅▅▄▄▄▃▄▄▃▄▄▄▄▄▃▂▅▅ █
  304 μs        Histogram: log(frequency) by time       1.55 ms <

 Memory estimate: 423.42 KiB, allocs estimate: 4083.

In [27]:
@benchmark Conv_backward_v5(input, weights, bias, gradient)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  127.000 μs …  2.742 ms  ┊ GC (min … max): 0.00% … 91.59%
 Time  (median):     133.200 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   153.003 μs ± 71.569 μs  ┊ GC (mean ± σ):  1.10% ±  2.74%

  ██▆▅▄▄▄▃▃▃▂▂▂▁▁▁▁                          ▁▁▂▂▂▁▁           ▂
  █████████████████████▇▇▆▇▇▇▇▇▆▇▇▆▇▇▇▆▇▇▇▇████████████▇▇▇▇▅▆▆ █
  127 μs        Histogram: log(frequency) by time       289 μs <

 Memory estimate: 43.27 KiB, allocs estimate: 28.

In [23]:
Conv_backward_v4(input, weights, bias, gradient)

(Float32[1.0270078 2.6579611 … 3.7924967 1.8244433; 2.597403 5.3894105 … 6.0019026 4.2386637; … ; 2.756383 5.254015 … 5.7856393 3.91924; 1.7586558 3.7111154 … 3.766919 2.8490338;;;], Float32[164.96873 172.18361 170.53923; 170.70602 169.33534 168.48164; 175.28775 170.91432 169.54118;;;; 170.53622 165.45215 168.81535; 171.62535 171.87164 168.82399; 174.88133 171.7694 171.18498;;;; 166.93677 168.05923 167.06853; 167.62666 167.41205 169.69621; 170.73174 168.36333 169.48141;;;; 161.80113 155.83371 156.12274; 162.93762 157.15332 157.87662; 161.28285 163.23285 162.17902;;;; 167.1234 168.40689 166.27711; 168.05473 168.79185 167.55911; 170.86842 170.41064 175.09317;;;; 164.01128 162.60814 159.25778; 157.55978 158.29161 166.03732; 160.68312 161.62408 166.63004], [348.5285866686615, 350.60216179457456, 343.41925508530653, 327.5206907433597, 346.788425115532, 332.3772648887847])

In [22]:
Conv_backward_v5(input, weights, bias, gradient)

(Float32[1.0270078 2.6579611 … 3.7924964 1.8244432; 2.5974026 5.3894105 … 6.001902 4.2386637; … ; 2.7563832 5.254015 … 5.7856393 3.9192402; 1.7586558 3.7111154 … 3.7669184 2.849034;;;], Float32[164.96873 172.18361 170.53923; 170.70602 169.33534 168.48164; 175.28775 170.91432 169.54118;;;; 170.53622 165.45215 168.81535; 171.62535 171.87164 168.82399; 174.88133 171.7694 171.18498;;;; 166.93677 168.05923 167.06853; 167.62666 167.41205 169.69621; 170.73174 168.36333 169.48141;;;; 161.80113 155.83371 156.12274; 162.93762 157.15332 157.87662; 161.28285 163.23285 162.17902;;;; 167.1234 168.40689 166.27711; 168.05473 168.79185 167.55911; 170.86842 170.41064 175.09317;;;; 164.01128 162.60814 159.25778; 157.55978 158.29161 166.03732; 160.68312 161.62408 166.63004], [348.5285866686615, 350.60216179457456, 343.41925508530653, 327.5206907433597, 346.788425115532, 332.3772648887847])